# 02 — Modelos de CF Explícito
Se entrena un baseline de matriz de factores (ALS/Autoencoder).

In [ ]:
# Este es un stub de flujo. Reemplazar con ALS real (implicit/surprise) o Autoencoder TF.
import yaml, numpy as np, scipy.sparse as sp
from src.models.cf_explicit import MFExplicitALSStub

with open('config/config.yaml') as f:
    cfg = yaml.safe_load(f)

# Construir matriz usuario-item de ejemplo (reemplazar por rating_complete)
n_users, n_items = 1000, 2000
rows = np.random.randint(0, n_users, size=10000)
cols = np.random.randint(0, n_items, size=10000)
data = np.random.randint(1, 11, size=10000).astype(float)
R = sp.csr_matrix((data, (rows, cols)), shape=(n_users, n_items))

model = MFExplicitALSStub(n_factors=64)
model.fit(R)
print('Predicción ejemplo u0,i0:', model.predict(0,0))
